# Hands on Object Relational Mapping in PyCram

This tutorial will walk through the serialization of a minimal plan in pycram.
First we will import sqlalchemy, create an in memory database and connect a session to it.

In [1]:
import sqlalchemy
import sqlalchemy.orm

engine = sqlalchemy.create_engine("sqlite+pysqlite:///:memory:", echo=False)
session = sqlalchemy.orm.Session(bind=engine)
session

Next we create the database schema using the sqlalchemy functionality. For that we need to import the base class of pycram.orm.

In [2]:
import pycram.orm.base
import pycram.orm.task
import pycram.orm.object_designator
import pycram.orm.motion_designator
import pycram.orm.action_designator
pycram.orm.base.Base.metadata.create_all(engine)
session.commit()

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


Next we will write a simple plan where the robot parks his arms and then moves somewhere. We will construct a TaskTree around it such that we can serialize it later. As usual, we first create a world and then define the plan. After that we get and print the task tree.

In [3]:
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.process_module import simulated_robot
from pycram.enums import Arms
from pycram.task import with_tree
import pycram.task
from pycram.bullet_world import BulletWorld, Object
from pycram.robot_descriptions import robot_description
from pycram.designators.object_designator import *
import anytree

world = BulletWorld()
pr2 = Object("pr2", "robot", "pr2.urdf")
kitchen = Object("kitchen", "environment", "kitchen.urdf")
milk = Object("milk", "milk", "milk.stl", position=[1.3, 1, 0.9])
cereal = Object("cereal", "cereal", "breakfast_cereal.stl", position=[1.3, 0.7, 0.95])
milk_desig = ObjectDesignatorDescription(names=["milk"])
cereal_desig = ObjectDesignatorDescription(names=["cereal"])
robot_desig = ObjectDesignatorDescription(names=["pr2"]).resolve()
kitchen_desig = ObjectDesignatorDescription(names=["kitchen"])

@with_tree
def plan():
    with simulated_robot:
        ParkArmsAction.Action(Arms.BOTH).perform()
        MoveTorsoAction([0.3]).resolve().perform()
        pickup_pose = CostmapLocation(target=cereal_desig.resolve(), reachable_for=robot_desig).resolve()
        pickup_arm = pickup_pose.reachable_arms[0]
        NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()
        PickUpAction(object_designator_description=cereal_desig, arms=[pickup_arm], grasps=["front"]).resolve().perform()
        ParkArmsAction([Arms.BOTH]).resolve().perform()

        place_island = SemanticCostmapLocation("kitchen_island_surface", kitchen_desig.resolve(),
                                           cereal_desig.resolve()).resolve()

        place_stand = CostmapLocation(place_island.pose, reachable_for=robot_desig, reachable_arm=pickup_arm).resolve()

        NavigateAction(target_locations=[place_stand.pose]).resolve().perform()

        PlaceAction(cereal_desig, target_locations=[place_island.pose], arms=[pickup_arm]).resolve().perform()

        ParkArmsAction.Action(Arms.BOTH).perform()

plan()

# set description of what we are doing
pycram.orm.base.MetaData().description = "Tutorial for getting familiar with the ORM."
task_tree = pycram.task.task_tree
print(anytree.RenderTree(task_tree))

no_operation()
└── plan()
    ├── perform(ParkArmsAction, )
    ├── perform(MoveTorsoAction, )
    ├── perform(NavigateAction, )
    ├── perform(PickUpAction, )
    ├── perform(ParkArmsAction, )
    ├── perform(NavigateAction, )
    ├── perform(PlaceAction, )
    └── perform(ParkArmsAction, )


Next we serialize the task tree by just recursively inserting from its root and close the simulation.

In [4]:
task_tree.root.insert(session)
world.exit()

Inserting TaskTree into database: 100%|██████████| 10/10 [00:00<00:00, 72.24it/s]


We can look at our experiment MetaData to get some context on the data we just created.

In [5]:
print(*session.query(pycram.orm.base.MetaData).all())

pycram.orm.base.MetaData(2023-06-21 11:11:52, tom_sch, Tutorial for getting familiar with the ORM., 3b4179f690079c2f066b528366d3831e85a5c841, 1, None)


Lastly we can look at various table to see how the structures got logged.
For example, we can get all the navigate actions that occurred.

In [6]:
navigations = session.query(pycram.orm.action_designator.NavigateAction).all()
print(*navigations, sep="\n")

pycram.orm.action_designator.NavigateAction(3, Navigate, 3, 1, 3, 3)
pycram.orm.action_designator.NavigateAction(6, Navigate, 6, 1, 8, 8)


Since inheritance is correctly mapped in the ORM package we can also get all actions that were executed with the correct classes in just one line.

In [7]:
actions = session.query(pycram.orm.action_designator.Action).all()
print(*actions, sep="\n")

pycram.orm.action_designator.ParkArmsAction(1, ParkArms, 1, 1, BOTH)
pycram.orm.action_designator.MoveTorsoAction(2, MoveTorso, 2, 1, 0.3)
pycram.orm.action_designator.NavigateAction(3, Navigate, 3, 1, 3, 3)
pycram.orm.action_designator.PickUpAction(4, PickUp, 4, 1, left, front, 1)
pycram.orm.action_designator.ParkArmsAction(5, ParkArms, 5, 1, BOTH)
pycram.orm.action_designator.NavigateAction(6, Navigate, 6, 1, 8, 8)
pycram.orm.action_designator.PlaceAction(7, Place, 7, 1, left, 12, 12, 2)
pycram.orm.action_designator.ParkArmsAction(8, ParkArms, 8, 1, BOTH)


Of course all relational algebra operators, such as filtering and joining also work in pycram.orm queries. For example if we want all actions that have an object designator assigned in them, we can execute:

In [8]:
object_actions = session.query(pycram.orm.action_designator.PickUpAction,
                               pycram.orm.object_designator.ObjectDesignator).join(pycram.orm.object_designator.ObjectDesignator).all() + \
                 session.query(pycram.orm.action_designator.PlaceAction,
                               pycram.orm.object_designator.ObjectDesignator).join(pycram.orm.object_designator.ObjectDesignator).all()
print(*object_actions, sep="\n")

(pycram.orm.action_designator.PickUpAction(4, PickUp, 4, 1, left, front, 1), pycram.orm.object_designator.ObjectDesignator(1, Object, cereal, cereal, 6, 6, 1))
(pycram.orm.action_designator.PlaceAction(7, Place, 7, 1, left, 12, 12, 2), pycram.orm.object_designator.ObjectDesignator(2, Object, cereal, cereal, 11, 11, 1))


If we want to filter for all successful tasks we can just add the filter operator.

In [9]:
successful_tasks = session.query(pycram.orm.task.TaskTreeNode).filter(pycram.orm.task.TaskTreeNode.status == "SUCCEEDED")
print(*successful_tasks, sep="\n")

pycram.orm.task.TaskTreeNode(2, 2, 2023-06-21 13:11:48.360113, 2023-06-21 13:11:52.938849, SUCCEEDED, None, 1, 1)
pycram.orm.task.TaskTreeNode(3, 3, 2023-06-21 13:11:48.360178, 2023-06-21 13:11:48.861320, SUCCEEDED, None, 2, 1)
pycram.orm.task.TaskTreeNode(4, 4, 2023-06-21 13:11:48.861418, 2023-06-21 13:11:49.362737, SUCCEEDED, None, 2, 1)
pycram.orm.task.TaskTreeNode(5, 5, 2023-06-21 13:11:49.681729, 2023-06-21 13:11:50.183847, SUCCEEDED, None, 2, 1)
pycram.orm.task.TaskTreeNode(6, 6, 2023-06-21 13:11:50.184025, 2023-06-21 13:11:50.692115, SUCCEEDED, None, 2, 1)
pycram.orm.task.TaskTreeNode(7, 7, 2023-06-21 13:11:50.692217, 2023-06-21 13:11:51.194235, SUCCEEDED, None, 2, 1)
pycram.orm.task.TaskTreeNode(8, 8, 2023-06-21 13:11:51.424789, 2023-06-21 13:11:51.926648, SUCCEEDED, None, 2, 1)
pycram.orm.task.TaskTreeNode(9, 9, 2023-06-21 13:11:51.926780, 2023-06-21 13:11:52.435604, SUCCEEDED, None, 2, 1)
pycram.orm.task.TaskTreeNode(10, 10, 2023-06-21 13:11:52.435685, 2023-06-21 13:11:52.938

As expected all but the root node succeeded, since the root node is still running.

Writing an extension to the ORM package is also done with ease. We need to create a class that can be inserted and its ORM equivalent, write a to_sql() and insert() method and then insert it somewhere.

In [10]:
import pycram.designators.action_designator


# define ORM class from pattern in every pycram.orm class
class ORMSaying(pycram.orm.action_designator.Action):
    __tablename__ = "Saying"
    id = sqlalchemy.Column(sqlalchemy.types.Integer, sqlalchemy.ForeignKey("Action.id"), primary_key=True)
    text = sqlalchemy.Column(sqlalchemy.types.String(255))

    __mapper_args__ = {
        "polymorphic_identity": __tablename__,
        "polymorphic_on": "dtype",
    }

    def __init__(self, text: str):
        super().__init__()
        self.text = text


# define brand new action designator
class Saying(pycram.designators.action_designator.ActionDesignatorDescription):

    def __init__(self, text):
        super().__init__()
        self.text = text

    def to_sql(self):
        return ORMSaying(self.text)

    def insert(self, session):
        action = self.to_sql()
        session.add(action)
        session.commit()
        return action

So we now got our new ActionDesignator called Saying and its ORM version. Since this class got created after all other classes got inserted into the database (in the beginning of the notebook) we have to insert it manually.

In [11]:
ORMSaying.metadata.create_all(bind=engine)

Now we can create and insert a Saying action.

In [12]:
# create a saying action and insert it
foo = Saying("Patchie, Patchie; Where is my Patchie?")
orm_foo = foo.insert(session)
orm_foo

__main__.ORMSaying(9, Saying, None, None, Patchie, Patchie; Where is my Patchie?)

It is notable that committing the object to the session fills its primary key. Hence, there is no worries about assigning unique IDs manually.
Finally, we can double-check that our object exists in the database.

In [13]:
session.query(ORMSaying).all()

[__main__.ORMSaying(9, Saying, None, None, Patchie, Patchie; Where is my Patchie?)]